<a href="https://colab.research.google.com/github/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/blob/main/Code/New_Notebook_Blancpain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DMML TEAM BLANCPAIN

##Import some librairies

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")
from sklearn.model_selection import train_test_split

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##Data Importation##

In [ ]:
df_train = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/training_data.csv')

df_test = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/test_data.csv')

df_sample = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/sample_submission.csv')

##Data visualization##

###Training set

In [ ]:
df_train.head()

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1


In [ ]:
print("There are {0} rows and {1} columns in the train dataset.".format(df_train.shape[0],df_train.shape[1]))

There are 6471 rows and 5 columns in the train dataset.


###Test Set

In [ ]:
df_test.head()

,id,keyword,location,text
0,9972,tsunami,NaN,Crptotech tsunami and banks.\n http://t.co/KHz...
1,9865,traumatised,"Portsmouth, UK",I'm that traumatised that I can't even spell p...
2,1937,burning%20buildings,NaN,@foxnewsvideo @AIIAmericanGirI @ANHQDC So ... ...
3,3560,desolate,NaN,Me watching Law &amp; Order (IB: @sauldale305)...
4,2731,crushed,bahstun/porta reeko,Papi absolutely crushed that ball


In [ ]:
print("There are {0} rows and {1} columns in the test dataset.".format(df_test.shape[0],df_test.shape[1]))

There are 1142 rows and 4 columns in the test dataset.


###Base Rate 

In [ ]:
base_rate = max(len(df_train[df_train["target"] == 0]) / len(df_train), len(df_train[df_train["target"] == 1]) / len(df_train))
print("The base rate for this problem is :", base_rate)

The base rate for this problem is : 0.5719363313243703


##Processing

In [ ]:
df_train["keyword_text"] = df_train['keyword'] + ' ' + df_train['text'] 
df_train["keyword_text"] = df_train["keyword_text"].astype(str)
df_test["keyword_text"] = df_test['keyword'] + ' ' + df_test['text'] 
df_test["keyword_text"] = df_test["keyword_text"].astype(str)

In [ ]:
df_train.head()

,id,keyword,location,text,target,keyword_text
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0,destroyed Black Eye 9: A space battle occurred...
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0,bioterror #world FedEx no longer to transport ...
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1,windstorm Reality Training: Train falls off el...
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1,hazardous #Taiwan Grace: expect that large roc...
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1,hostage New ISIS Video: ISIS Threatens to Behe...


In [ ]:
df_train_train, df_train_test = train_test_split(df_train, test_size=0.2, random_state = 50)

###With "Keyword" feature

In [ ]:
label = LabelEncoder()
label_keyword_train = pd.Series(label.fit_transform(df_train_train["keyword"]), name = "keyword")
label_keyword_test = pd.Series(label.fit_transform(df_train_test["keyword"]), name = "keyword")

TypeError: ignored

In [ ]:
df_train_train.dtypes

id               int64
keyword         object
location        object
text            object
target           int64
keyword_text    object
dtype: object

In [ ]:
df_train_train